In [1]:
import gymnasium as gym
from gymnasium import envs

# List all registered environments (including Atari)
all_envs = envs.registry.keys()
atari_envs = [env_id for env_id in all_envs if "ALE/" in env_id]

# Print first 10 Atari games
print(list(atari_envs)[:10])  # Should include 'ALE/Frogger-v5'

[]


In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
import cv2
import time
import warnings
import ale_py
import gymnasium as gym
import multiprocessing as mp
from datetime import datetime
import numpy as np
import os
import pandas as pd

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
'''
import gymnasium as gym

# Modern way to create Atari environment
env = gym.make("ALE/Frogger-v5", render_mode="human")  # Note: ALE/ prefix required


observation, info = env.reset()
for _ in range(100):
    action = env.action_space.sample()  # Random action
    observation, reward, terminated, truncated, info = env.step(action)
        
    if terminated or truncated:
        observation, info = env.reset()
'''

'\nimport gymnasium as gym\n\n# Modern way to create Atari environment\nenv = gym.make("ALE/Frogger-v5", render_mode="human")  # Note: ALE/ prefix required\n\n\nobservation, info = env.reset()\nfor _ in range(100):\n    action = env.action_space.sample()  # Random action\n    observation, reward, terminated, truncated, info = env.step(action)\n        \n    if terminated or truncated:\n        observation, info = env.reset()\n'

In [4]:
# Cell 2: Define the environment runner function
def run_frogger_instance(instance_id, num_episodes=5, render_mode=None):
    """
    Runs a single instance of the Frogger environment
    
    Args:
        instance_id (int): Identifier for this process
        num_episodes (int): Number of episodes to run
        render_mode (str): None, "human", or "rgb_array"
    """
    env = gym.make("ALE/Frogger-v5", render_mode=render_mode)
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            # Replace this with your actual policy/controller
            action = env.action_space.sample()  # Random actions for demo
            
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            total_reward += reward
            
            # Add your custom processing here
            
        print(f"Instance {instance_id} | Episode {episode+1}/{num_episodes} | Reward: {total_reward}")
    
    env.close()

In [5]:
# Cell 3: Main execution block (must be in its own cell)
if __name__ == "__main__":  # Critical for multiprocessing in Jupyter
    # Configuration
    NUM_INSTANCES = 4       # Number of parallel instances
    EPISODES_PER_INSTANCE = 3
    RENDER_MODE = None      # Set to "human" if you want visualization
    
    print(f"Starting {NUM_INSTANCES} parallel Frogger instances at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = datetime.now()

Starting 4 parallel Frogger instances at 2025-04-09 09:23:57


In [6]:
   # Create and start processes
processes = []
for i in range(NUM_INSTANCES):
        # Only render the first instance if RENDER_MODE is "human"
    current_render_mode = RENDER_MODE if i == 0 else None
        
    p = mp.Process(
        target=run_frogger_instance,
        args=(i, EPISODES_PER_INSTANCE, current_render_mode)
        )
    p.start()
    processes.append(p)
    
    # Wait for all processes to complete
    for p in processes:
        p.join()
    
    duration = (datetime.now() - start_time).total_seconds()
    print(f"All instances completed in {duration:.2f} seconds")

All instances completed in 0.07 seconds
All instances completed in 0.13 seconds
All instances completed in 0.19 seconds
All instances completed in 0.25 seconds


In [7]:
import gymnasium as gym
import multiprocessing as mp
import numpy as np

def run_env(env_name, num_episodes, process_id):
    env = gym.make(env_name, render_mode="human")  # or "rgb_array" for no rendering
    for episode in range(num_episodes):
        obs, info = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()  # Random policy - replace with your agent
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            # Add your learning/processing here
        print(f"Process {process_id} finished episode {episode}")
    env.close()

if __name__ == "__main__":
    env_name = "ALE/Frogger-v5"  # Or the correct Frogger environment name
    num_processes = 4  # Number of parallel instances
    num_episodes = 10  # Episodes per instance
    
    processes = []
    for i in range(num_processes):
        p = mp.Process(target=run_env, args=(env_name, num_episodes, i))
        p.start()
        processes.append(p)
    
    for p in processes:
        p.join()

In [ ]:
import numpy as np
import gymnasium as gym
from keras.models import Sequential, clone_model
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
import time
import matplotlib.pyplot as plt
from collections import deque
import random
import math
import multiprocessing as mp
from datetime import datetime

# --- Hyperparameters ---
EPISODES = 1000                # Total training episodes
BATCH_SIZE = 32                # Batch size for training
MEMORY_SIZE = 100000           # Larger replay memory (for PER)
GAMMA = 0.99                   # Higher discount factor
EPSILON_START = 1.0            # Starting exploration rate
EPSILON_MIN = 0.01             # Min exploration rate
EPSILON_DECAY = 0.995          # Decay rate
LEARNING_RATE = 0.00025        # Lower learning rate (better stability)
UPDATE_TARGET_FREQ = 1000      # Double DQN target network update
FRAME_STACK = 4                # Number of stacked frames (temporal info)
SAVE_FREQ = 50                 # Save weights every N episodes
NUM_ENVS = 4                   # Number of parallel environments

# --- Prioritized Experience Replay (PER) ---
class PrioritizedReplayBuffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)
        self.priorities = deque(maxlen=max_size)
    
    def add(self, state, action, reward, next_state, done):
        max_priority = max(self.priorities) if self.buffer else 1.0
        self.buffer.append((state, action, reward, next_state, done))
        self.priorities.append(max_priority)
    
    def sample(self, batch_size, alpha=0.6):
        priorities = np.array(self.priorities)
        probs = priorities ** alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[i] for i in indices]
        weights = (len(self.buffer) * probs[indices]) ** (-0.4)
        weights /= weights.max()
        
        states = np.array([x[0] for x in samples])
        actions = np.array([x[1] for x in samples])
        rewards = np.array([x[2] for x in samples])
        next_states = np.array([x[3] for x in samples])
        dones = np.array([x[4] for x in samples])
        
        return states, actions, rewards, next_states, dones, indices, weights
    
    def update_priorities(self, indices, errors, offset=0.01):
        for i, error in zip(indices, errors):
            self.priorities[i] = abs(error) + offset

# --- Environment Worker ---
class EnvironmentWorker(mp.Process):
    def __init__(self, task_queue, result_queue, model_weights, epsilon):
        super().__init__()
        self.task_queue = task_queue
        self.result_queue = result_queue
        self.model_weights = model_weights
        self.epsilon = epsilon
        
    def run(self):
        # Create local environment and model
        env = gym.make("ALE/Frogger-v5", render_mode="rgb_array")
        model = build_model()
        model.set_weights(self.model_weights)
        frame_stacker = FrameStacker()
        
        while True:
            task = self.task_queue.get()
            if task is None:  # Termination signal
                env.close()
                break
                
            # Get initial state
            state, _ = env.reset()
            state = preprocess_state(state)
            state = frame_stacker.reset(state)
            done = False
            total_reward = 0
            
            while not done:
                # Epsilon-greedy action
                if np.random.rand() <= self.epsilon:
                    action = env.action_space.sample()
                else:
                    q_values = model.predict(np.expand_dims(state, axis=0), verbose=0)
                    action = np.argmax(q_values[0])
                
                next_state, reward, done, _, _ = env.step(action)
                next_state = preprocess_state(next_state)
                next_state = frame_stacker.append(next_state)
                total_reward += reward
                
                # Send experience to main process
                self.result_queue.put((state, action, reward, next_state, done))
                state = next_state
            
            # Send episode results
            self.result_queue.put(('episode_result', total_reward))

# --- Model Definition ---
def build_model():
    model = Sequential([
        Conv2D(32, (8, 8), strides=4, activation="relu", input_shape=(84, 84, FRAME_STACK)),
        Conv2D(64, (4, 4), strides=2, activation="relu"),
        Conv2D(64, (3, 3), strides=1, activation="relu"),
        Flatten(),
        Dense(512, activation="relu"),
        Dense(env.action_space.n, activation="linear")
    ])
    model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE))
    return model

# --- Frame Stacking ---
class FrameStacker:
    def __init__(self):
        self.frames = deque(maxlen=FRAME_STACK)
    
    def reset(self, state):
        for _ in range(FRAME_STACK):
            self.frames.append(state)
        return np.stack(self.frames, axis=-1)
    
    def append(self, state):
        self.frames.append(state)
        return np.stack(self.frames, axis=-1)

# --- Preprocessing ---
def preprocess_state(state):
    state = np.mean(state, axis=2)  # Grayscale
    state = state[34:194, :]        # Crop
    state = state[::2, ::2]         # Downsample to 80x80
    state = np.pad(state, ((2,2),(2,2)), mode='constant')
    return state / 255.0

# --- Main Training Function ---
def train_dqn_parallel():
    # Initialize main model and target model
    main_model = build_model()
    target_model = clone_model(main_model)
    target_model.set_weights(main_model.get_weights())
    
    # Create shared memory for model weights
    shared_weights = mp.Array('d', main_model.get_weights()[0].flatten())
    
    # Create queues and workers
    task_queue = mp.Queue()
    result_queue = mp.Queue()
    workers = []
    
    for _ in range(NUM_ENVS):
        worker = EnvironmentWorker(task_queue, result_queue, main_model.get_weights(), EPSILON_START)
        worker.start()
        workers.append(worker)
    
    # Initialize replay memory
    memory = PrioritizedReplayBuffer(MEMORY_SIZE)
    rewards_history = []
    global_step = 0
    epsilon = EPSILON_START
    start_time = time.time()
    
    # Start episodes
    for episode in range(1, EPISODES + 1):
        # Send tasks to workers
        for _ in range(NUM_ENVS):
            task_queue.put(('start_episode',))
        
        # Collect experiences and results
        episode_rewards = []
        for _ in range(NUM_ENVS):
            while True:
                result = result_queue.get()
                if result[0] == 'episode_result':
                    episode_rewards.append(result[1])
                    break
                else:
                    state, action, reward, next_state, done = result
                    memory.add(state, action, reward, next_state, done)
        
        # Train on collected experiences
        if len(memory.buffer) >= BATCH_SIZE:
            states, actions, rewards, next_states, dones, indices, weights = memory.sample(BATCH_SIZE)
            
            # Double DQN update
            next_q_values = target_model.predict(next_states, verbose=0)
            best_actions = np.argmax(main_model.predict(next_states, verbose=0), axis=1)
            target_q = rewards + GAMMA * next_q_values[np.arange(BATCH_SIZE), best_actions] * (1 - dones)
            
            # Update priorities
            current_q = main_model.predict(states, verbose=0)
            td_errors = target_q - current_q[np.arange(BATCH_SIZE), actions]
            memory.update_priorities(indices, td_errors)
            
            # Train main model
            target = current_q.copy()
            target[np.arange(BATCH_SIZE), actions] = target_q
            main_model.fit(states, target, sample_weight=weights, epochs=1, verbose=0)
            
            global_step += 1
            
            # Update target network
            if global_step % UPDATE_TARGET_FREQ == 0:
                target_model.set_weights(main_model.get_weights())
        
        # Update epsilon
        if epsilon > EPSILON_MIN:
            epsilon *= EPSILON_DECAY
        
        # Update workers with new epsilon and weights
        for worker in workers:
            worker.epsilon = epsilon
        
        # Update shared weights
        main_model.save_weights('temp_weights.h5')
        new_weights = main_model.get_weights()
        for worker in workers:
            worker.model_weights = new_weights
        
        # Record and print progress
        avg_reward = np.mean(episode_rewards)
        rewards_history.append(avg_reward)
        
        elapsed_time = (time.time() - start_time) / 60
        remaining_time = (elapsed_time / episode) * (EPISODES - episode)
        print(f"Episode: {episode}/{EPISODES}, Avg Reward: {avg_reward:.2f}, Epsilon: {epsilon:.2f}, Time: {elapsed_time:.2f} mins")
        
        # Save weights periodically
        if episode % SAVE_FREQ == 0:
            main_model.save_weights(f"frogger_parallel_ep{episode}.weights.h5")
    
    # Clean up workers
    for _ in range(NUM_ENVS):
        task_queue.put(None)
    for worker in workers:
        worker.join()
    
    # Plot and save results
    plt.plot(rewards_history)
    plt.xlabel("Episode")
    plt.ylabel("Average Reward")
    plt.title("Parallel DQN Training Progress")
    plt.savefig("frogger_parallel_training.png")
    plt.show()
    
    return main_model

# --- Run Training ---
if __name__ == "__main__":
    env = gym.make("ALE/Frogger-v5")  # For action space reference
    model = train_dqn_parallel()
    model.save_weights("frogger_parallel_final.weights.h5")
    env.close()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
